In [85]:
import pandas as pd
import numpy as np
import os
import praw
from dotenv import load_dotenv

In [86]:
df = pd.read_csv('../datasets/all_train.tsv', sep='\t')
df.head()

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
0,0,0,NaN,NaN,Alexithymia,my walgreens offbrand mucinex was engraved wit...,1.551641e+09,i.imgur.com,True,awxhir,https://external-preview.redd.it/WylDbZrnbvZdB...,NaN,2.0,12,mildlyinteresting,My Walgreens offbrand Mucinex was engraved wit...,0.84,1,0,0
1,1,1,155885.0,714550.0,RickSisco,NaN,1.443822e+09,NaN,True,cvm5uy4,http://i.imgur.com/yxrkYT8.jpg,3n7fld,NaN,5,psbattle_artwork,NaN,NaN,0,2,4
2,2,2,NaN,NaN,VIDCAs17,this concerned sink with a tiny hat,1.534727e+09,i.redd.it,True,98pbid,https://preview.redd.it/wsfx0gp0f5h11.jpg?widt...,NaN,2.0,119,pareidolia,This concerned sink with a tiny hat,0.99,0,2,2
3,3,3,NaN,NaN,prometheus1123,hackers leak emails from uae ambassador to us,1.496511e+09,aljazeera.com,True,6f2cy5,https://external-preview.redd.it/6fNhdbc6K1vFA...,NaN,1.0,44,neutralnews,Hackers leak emails from UAE ambassador to US,0.92,1,0,0
4,4,4,282323.0,1228398.0,NaN,NaN,1.378792e+09,NaN,True,cc5cbon,http://i.imgur.com/M8KTWMx.jpg,1lz1q0,NaN,3,psbattle_artwork,NaN,NaN,0,2,4


In [57]:
df_filtered = df[df['2_way_label'] == 1]


In [87]:
df=df[['clean_title','id','author','linked_submission_id','subreddit','2_way_label','6_way_label']]

In [88]:
df.dropna(subset=['id'], inplace=True)

In [90]:
print(df['6_way_label'].value_counts())

6_way_label
0    400433
4    242100
2    142939
1     42332
5     26584
3     23830
Name: count, dtype: int64


In [91]:
load_dotenv()

# Get credentials from environment
client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_CLIENT_SECRET")
user_agent = os.getenv("REDDIT_AGENT")

# Set up Reddit API with PRAW
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

def fetch_submission_content(submission_id):
    try:
        submission = reddit.submission(id=submission_id.replace("t3_", ""))
        content = f"{submission.title}\n\n{submission.selftext}"
        link = f"https://www.reddit.com{submission.permalink}"
        return content, link
    except Exception as e:
        print(f"Error fetching submission {submission_id}: {e}")
        return None, None

In [ ]:
df.loc[df['id'].notna(), 'content'] = df.loc[
    df['id'].notna(), 'linked_submission_id'
].apply(lambda sid: fetch_submission_content(sid)[0])


KeyboardInterrupt: 

,clean_title,author,linked_submission_id,subreddit,2_way_label,content
0,my walgreens offbrand mucinex was engraved wit...,Alexithymia,NaN,mildlyinteresting,1,my walgreens offbrand mucinex was engraved wit...
2,this concerned sink with a tiny hat,VIDCAs17,NaN,pareidolia,0,this concerned sink with a tiny hat
3,hackers leak emails from uae ambassador to us,prometheus1123,NaN,neutralnews,1,hackers leak emails from uae ambassador to us
5,this flower in my neighborhood,NaN,NaN,mildlyinteresting,1,this flower in my neighborhood
6,puppy taking in the view,NaN,NaN,photoshopbattles,1,puppy taking in the view


In [31]:
print(df["content"].iloc[3])


Chipmunk-faced while weightlifting


